In [1]:
using DataFrames, DataFramesMeta
using Plots
using Distributions
using StatsModels
gr()

Plots.GRBackend()

In [2]:
using ClobberingReload

In [3]:
using StatPlots

# Exploratory Analysis
Some things we need to do:
- set a 'good/not good' column
- empirical bayes for estimation of movie rating
- perform joins and see most successful actors, producers, writers, directors
- compare tastes of public vs tastes of critics

more?

## Loading the data, adding some calculated columns

In [4]:
basic_cols = [:id, :title, :metascore, :user_score, :release_date, :running_time, :rating, :company, :positive, :mixed, :negative]
review_cols = [:id, :score, :publication, :critic]
basics = readtable("../data/basics.csv", names = basic_cols,header=false);
reviews = readtable("../data/reviews.csv", names=review_cols, header=false);

In [5]:
string_to_float(str) = try parse(Float64, str) catch return(NA) end
basics[:user_score] = map(string_to_float, basics[:user_score]);

We now add a thumbs up/thumbs down column. The idea is that we are mostly interested in the 'good' movies, and to preserve generalization, we keep to an idea of 'good' that is very general. We don't want to make a distinction in our algorithm between excellent, very good, and good. We just want to recommend movies that you'll be happy watching 

In [6]:
function score_to_star(sc::Int64)
    return max(1, convert(Int64, round(sc/10)) )
end

score_to_star (generic function with 1 method)

In [7]:
cutoff = 70 # this could be changed
reviews_normalized = reviews[:score] .>= cutoff;
reviews[:stars] = score_to_star.(convert(Array{Int64, 1}, reviews[:score]));

In [8]:
reviews[:thumbsup] = reviews_normalized;

We also want to change the 'Staff (uncredited)' values to the name of the publication.

In [9]:
reviews[:critic_clean] = reviews[:critic]
for (index, critic) in enumerate(reviews[:critic_clean])
    if contains(critic, "Staff")
        reviews[:critic_clean][index] = reviews[:publication][index]
    end
end

some quick and dirty empirical bayes to produce better rating averages per movie. We suppose the ratings follow a Dirichlet-Categorical distribution, and estimate the prior using the entire dataset.

In [10]:
include("../src/EmpiricalBayes.jl")
using EmpiricalBayes

EmpiricalBayes

In [12]:
# if we made some iterations on the model
# creload("../src/EmpiricalBayes.jl")

INFO: Reloading ../src/EmpiricalBayes.jl


"../src/EmpiricalBayes.jl"

In [56]:
categories = reviews[reviews[:id] .== "avatar", :][:stars]

35-element DataArrays.DataArray{Int64,1}:
 10
 10
 10
 10
 10
 10
 10
 10
 10
 10
  9
  9
  9
  ⋮
  8
  8
  8
  8
  8
  8
  7
  6
  6
  6
  5
  5

In [57]:
fit(Categorical, 10, categories )

Distributions.Categorical{Float64}(
K: 10
p: [0.0,0.0,0.0,0.0,0.0571429,0.0857143,0.0285714,0.257143,0.285714,0.285714]
)


In [32]:
model = fit_bayes(reviews, :id, :stars)sappi;
aggr = EmpiricalBayes.return_posterior(reviews, :id, :stars, model);

Let's add the average rating to the basics dataframe.

In [33]:
basics = join(basics, aggr[[:id, :adjusted_average]], on = :id, kind = :left);

Without further ado:
### The best movies according to metacritic critics:


In [16]:
basics = basics[isna(basics[:adjusted_average]) .== false,:]
sort!(basics, cols=[:adjusted_average], rev=true)
basics[1:20, [:id, :adjusted_average]]

,id,adjusted_average
1,boyhood,9.461665405254399
2,moonlight-2016,9.38102739004958
3,pans-labyrinth,9.195551965715314
4,manchester-by-the-sea,9.164145106291091
5,the-social-network,9.13127852510068
6,gravity,9.114046997566124
7,army-of-shadows,9.1087936475983
8,carol,9.06377206601342
9,4-months-3-weeks-and-2-days,9.050377400786404
10,ratatouille,9.026181639964921


Obviously this shows some biases. It looks like older movies are under-represented; this might be because some older movies have less critics? Let's check this later. 

First let's look at our score VS the user score:

In [17]:
basics_2 = basics[isna(basics[:user_score]) .== false, :];

In [34]:
date_to_oldnew(date)= parse(date[length(date)-3:length(date)]) > 2005
basics_2[:isnew] = map(date_to_oldnew, basics_2[:release_date]);

In [46]:
plot(basics_2, :adjusted_average, :adjusted_average, linewidth=2,
    linecolor = "gray", lab="x = y")
scatter!(basics_2, :adjusted_average, :user_score, group=:isnew,
    markersize = 2,
    markerstrokewidth = 0,
markeralpha = .5, lab=["old", "new"])

n = size(basics_2[:adjusted_average])[1]
bhat = [Array(basics_2[:adjusted_average]) ones(n)]\Array(basics_2[:user_score])

Plots.abline!(bhat..., linewidth = 1, linecolor = "navy", lab="regression")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 adjusted 
 
 
 average 
 
 
 user 
 
 
 score 
 
<polyline clip-path="url(#clip02)" style="stroke:#7f7f7f; stroke-width:1.6; stroke-opacity:1; fill:none" points="
 580.492,19.5331 574.724,22.5906 561.456,29.6231 559.209,30.8139 556.858,32.0601 555.625,32.7134 555.25,32.9126 552.029,34.6196 551.071,35.1275 549.34,36.0449 
 547.594,36.9704 546.019,37.8054 545.879,37.8797 545.839,37.9006 544.441,38.6419 544.441,38.6419 544.07,38.8382 543.479,39.1518 542.007,39.9317 541.899,39.9892 
 540.585,40.6854 540.341,40.8147 540.336,40.8177 540.188,40.8959 539.534,41.2424 538.835,41.6129 538.744,41.6612 538.12,41.9923 537.574,42.2813 
 536.823,42.6796 535.197,43.5415 534.395,43.9663 532.745,44.8413 531.978,45.2479 530.824,45.8593 530.304,46.1347 530.301,46.1364 529.411,46.6084 
 528.57,47.0538 528.102,47.3021 527.792,47.4665 527.167,47.7977 526.141,48.3416 525.955,48.4398 525.791,48.5272 525.595,48.631 525.504,48.6793 525.106,48.89 
 524.518,49.2016 524.49,49.2166 523.926,49.5157 523.395,49.7972 523.337,49.828 522.781,50.1226 522.257,50.4001 522.001,50.5361 521.889,50.5955 521.493,50.8054 
 521.007,51.0627 520.469,51.3478 519.709,51.7505 519.537,51.8418 519.287,51.9744 519.191,52.0251 519.158,52.0429 518.877,52.1915 
 518.875,52.1929 518.515,52.3834 518.466,52.4095 518.307,52.494 518.068,52.6207 517.972,52.6713 517.613,52.8615 517.596,52.8707 517.262,53.0476 
 517.188,53.0873 516.672,53.3605 516.455,53.4756 516.035,53.698 515.894,53.773 515.695,53.8784 515.238,54.1208 514.724,54.393 514.459,54.5333 514.028,54.7622 
 513.808,54.8787 513.406,55.0917 513.405,55.0922 513.213,55.1941 513.133,55.2365 512.934,55.342 512.879,55.3709 512.664,55.485 512.161,55.7514 
 512.118,55.7745 511.361,56.1756 511.083,56.3228 510.924,56.407 510.63,56.563 509.883,56.9588 509.532,57.1452 509.421,57.2037 509.299,57.2684 508.591,57.6438 
 508.561,57.6599 507.981,57.9672 507.858,58.0322 507.858,58.0322 507.603,58.1677 507.383,58.2841 507.139,58.4137 506.938,58.5202 506.817,58.584 
 506.567,58.7167 506.546,58.7277 505.964,59.0364 505.853,59.0951 505.358,59.3574 505.358,59.3574 505.272,59.4028 505.207,59.4374 505.114,59.4869 505.044,59.5241 
 505.044,59.5241 504.859,59.6221 504.792,59.6575 504.362,59.8855 504.144,60.0008 504.083,60.0333 503.965,60.0959 503.88,60.1408 503.788,60.1894 
 503.606,60.2862 503.27,60.4643 501.978,61.1489 501.796,61.2453 501.705,61.2936 501.529,61.3869 501.496,61.4044 501.184,61.5701 501.064,61.6333 
 500.964,61.6866 500.666,61.8446 500.568,61.8962 500.568,61.8962 500.426,61.9716 499.733,62.3392 499.718,62.347 499.677,62.3685 499.662,62.3768 499.64,62.3882 
 499.609,62.4047 499.227,62.6072 499.147,62.6497 498.404,63.0436 498.186,63.1591 498.075,63.2176 498.075,63.2176 497.951,63.2836 497.725,63.403 
 497.629,63.4542 497.423,63.5633 497.416,63.5671 497.265,63.6472 497.231,63.6651 497.174,63.6955 497.098,63.7354 497.015,63.7797 496.435,64.0873 
 496.424,64.0926 496.424,64.0926 496.175,64.2249 496.134,64.2468 496.07,64.2804 495.962,64.3376 495.773,64.4379 495.685,64.4845 495.685,64.4845 
 495.365,64.6543 495.26,64.7101 495.219,64.7318 495.098,64.7959 494.99,64.8529 494.871,64.9161 494.546,65.0884 494.267,65.236 493.969,65.3941 493.575,65.6032 
 493.529,65.6275 493.499,65.6435 493.499,65.6435 493.151,65.8278 492.716,66.0583 492.627,66.1052 492.496,66.1746 
 492.364,66.2447 492.364,66.2447 492.124,66.3723 492.053,66.4098 492.01,66.4327 491.841,66.522 491.632,66.6327 491.572,66.6649 491.344,66.7853 491.056,66.9382 
 490.941,66.9993 490.879,67.032 490.527,67.2183 490.493,67.2367 490.493,67.2367 490.466,67.2511 490.444,67.2627 490.159,67.4138 490.149,67.419 
 490.137,67.4255 490.12,67.434 490.12,67.434 490.098,67.4457 490.098,67.4457 489.926,67.5373 489.598,67.7107 489.414,67.8083 489.248,67.8964 489.133,67.9576 
 488.697,68.1883 488.647,68.215 488.559,68.2615 488.459,68.3147 488.29,68.40

We can clearly see some cool, expected features: users tend to like movies panned by critics *way more*, and users tend to be a little more disatisfied with moveies with high critical acclaim.

In [20]:
a = marginalhist(basics_2, :adjusted_average, :user_score, c=:matter)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 2.5 
 
 
 5.0 
 
 
 7.5 
 
 
 adjusted 
 
 
 average 
 
 
 user 
 
 
 score 
 
 
<image width="491" height="325" xlink:href="data:;base64,
iVBORw0KGgoAAAANSUhEUgAAAesAAAFFCAYAAADM5cU/AAAQP0lEQVR4nO3cvY9c93kF4Hd2ZsQd
krv8sEJKFC3KMuXAHzEURAHsBIGrGGlSBGDDQoXL/DUpA7h1wUZNCjdGkMRAgACJkTgGo4iIZMsK
KZqk+LVL7i5358OF7JRpzgXm9eJ5+nP3NzN35uxtzqjgNxb33lul1xhfvDYa4iysl3sBetlY9wEA
gP+fsgaA5pQ1ADSnrAGgOWUNAM0pawBoTlkDQHPKGgCaU9YA0JyyBoDmjs0cYDqPeBymEeOJyKOj
+Azjy9fX/j66F3rcC6nV/kF2gekkPsNoNovyy4eP4jNsbG9F+cOfvh/lX3r7a1F+COm9EN9LVTX9
5l+v9XfBkzUANKesAaA5ZQ0AzSlrAGhOWQNAc8oaAJpT1gDQnLIGgOaUNQA0p6wBoLnf+VnF31rc
vhHNK67m8+jvT954N34vj97/fvQalg+yacONrVNRvqpqcf9hdobzZ+IzHN78KMrv3fosys/ePBfl
q6oe/NP9KL/95okof/j4MMpXVf3D329H+e/+1V6U/+8fj6N8VdVikX2tz53J38dzry6i/Nm3s+/U
ar6M8lVVp//yT6L88m72fRh/+UqUr6oaTbL52nSK2ZM1ADSnrAGgOWUNAM0pawBoTlkDQHPKGgCa
U9YA0JyyBoDmlDUANKesAaA5c6PHyHJnN8uHk35VVRuvXojyq2fP4zMs7uSvI7GaZ/OQVVUvfp69
ho3NbBpxNMmnOlP3f5zNvm5/aRqf4Vc/zT7Lk2fyqc6zb2XTsc8+eRHlT17K38d0gnfzj78W5Ufn
z0b5qqrRbBblxxevmRsFgONMWQNAc8oaAJpT1gDQnLIGgOaUNQA0p6wBoDllDQDNKWsAaE5ZA0Bz
2SbhbyzuvRdNfaYzbFVV48vXo2vEc6XTfJJvee9BfI1EOhValc+FLu4/is+weJzNrqYml16Or7H5
9S9G+cOP7kb5Jz/Jpj6rqqZb630W2PnFUXyNn985nV3gTnyEuvp8L8q/lK2V1ku78+wCA1g+fhrl
x1vh51hVNcnvp4QnawBoTlkDQHPKGgCaU9YA0JyyBoDmlDUANKesAaA5ZQ0AzSlrAGhOWQNAc4PM
jQ4xF5pKJ09XO9lE5WiAudHRdjaJt9p5FuWXd+9H+aqqmmS31HInm1b8/AzjKH5wK5t9HX3yOMpX
Ve3fOYjymxfCjckB7N5eRvlHn2XfqQ8+PRnlq6om4ePM1Vfy+/nmR9nvwjvf3Iny8/1sibmqavJa
NsE7mm1mB5jmVbfaz76TKU/WANCcsgaA5pQ1ADSnrAGgOWUNAM0pawBoTlkDQHPKGgCaU9YA0Nwg
C2YdpCtqi9s3spmeo6Mo/vk15lk+XO6KV4IqX/nZ2M5Xp+Z3PovyJ14/E+WPHjyP8lVVJ6+civI7
t7IzPLid/zTce5itqN18nC2YXfv2kyhfVXXwPHueebabfSerql49exjlN8IjTLfyZ7pR+tu0md1L
o9ksyldVjd8IOyZc2fRkDQDNKWsAaE5ZA0BzyhoAmlPWANCcsgaA5pQ1ADSnrAGgOWUNAM0pawBo
7tjMjaZTbulcaDqzWVVV0+zjGIX5w5/eivJVVcvdvSg/Pr8Vn+HpT+5H+VNXszOs5osoX1V199+y
icmjF9m84+mt/DXce5jlP51n87v//O/ZbGxV1YPDaGGytvK10frOHzyN8qdfz2ZbT1w6HeU7WO3v
r/sIMU/WANCcsgaA5pQ1ADSnrAGgOWUNAM0pawBoTlkDQHPKGgCaU9YA0JyyBoDmsi29gcRToVU1
vngtei2L2zeiM6zCacQhrPaySb3Vs+fxGQ4/+Di+Rmq5m02/PvnJZ1F+upX/D/zDH25H+W+8ln+W
qd3n2fzt0/0sf/5U/p38ytvZd+ruh/mi8+t/mP08bl7J7qXRJN9MPfndb2Vn2A5niMM56aqqmmaz
rWlHebIGgOaUNQA0p6wBoDllDQDNKWsAaE5ZA0BzyhoAmlPWANCcsgaA5pQ1ADSXb+ENIJ1hq8rn
QtMpudXObpSvqqr5IoovHz+N8qv9bKazqmr62u9F+YObH8dn2Psoex8+vDmL8l/99osoX1U1GWW3
8807p6L8n349ew+rqj6+n72P/7GT/f1rF/OJyVNXs5nLr/7ZmfgM4/PZXOj4wvkoP5rkNbEK5z5H
Q8yFhoboqYQnawBoTlkDQHPKGgCaU9YA0JyyBoDmlDUANKesAaA5ZQ0AzSlrAGhOWQNAc4PMpy3u
vRdtIw4yNxqeocI5u9V8HuWrqlaPnkT5+S8/jfKHnzyI8lVVi93DKL8xy6cNP/i7vSi/WGS344vD
cZSvqnr9S/tR/sP/ORnlL34hn0z97PFLUf7lc9m99Nof5c8ik63sNQzh5DtvRvnRbDPKj6+8FuWH
kE6eji9fX+tU6BA8WQNAc8oaAJpT1gDQnLIGgOaUNQA0p6wBoDllDQDNKWsAaE5ZA0BzyhoAmsu3
HWuYudBYOBda02kUT6dCq6qWj59G+cmXX8/+/n4271hVdXT/bpSfh3OlVVWXfz+7Fzam2e38ox/N
onxVPjd69a1scvWVv3g1yldVvfHgWZTf2MxmMpcH+QTwyXeuxtdIpXOho9Onsnw49VlV8e8rnqwB
oD1lDQDNKWsAaE5ZA0BzyhoAmlPWANCcsgaA5pQ1ADSnrAGgufUvj1XV4vaNVXqN8eXr0WtJz7Dc
2U3iVVW1erKTXWAyjuLP//E/s79fVS998QtRfv+/sgW0qqqD+y/iayTSBbSqqvFm9n/07u1FlD/7
1okoX1U1vXAyyi/CNbvZ21eifFXVKPxOjS9dzM8QLphN3vreWn8bq/LfZzxZA0B7yhoAmlPWANCc
sgaA5pQ1ADSnrAGgOWUNAM0pawBoTlkDQHPKGgCaOzYTcPEk3nSa/f1f/m+Ur6oabWazggf/8rMo
v3frYZSvqnp+N5u5nGRvQVVVbX/jTJRf7GZzpY/fP4jyVVUXvvNylB9Nsv/DJxfORvmqqppOovhq
P/sclrt7Ub6q6sQ7X88uEM6VVlVtbG9lFwh/28YXrx2bnvhd5skaAJpT1gDQnLIGgOaUNQA0p6wB
oDllDQDNKWsAaE5ZA0BzyhoAmlPWANBctgfI/1k+ehpfYzV/FOXTichT03wacTzLXsNyvozPsPfR
TpQ/9+dvRflT3zoR5auqXty6HeXnj/ej/Ph8OHFZVdMvvx7lF5/ei/KT1y5E+aqqms+j+Gj7dHyE
8eXr0dxnOsW8uPdeNuVc658sPQ6vwZM1ADSnrAGgOWUNAM0pawBoTlkDQHPKGgCaU9YA0JyyBoDm
lDUANKesAaC5tc6n/dYQU3DLh9nM5epJNlE5On0qyldVHd36RXaGWTZzefTL+1G+qurgk2x2dTzL
J09PvvNmlH9x606Un73zlShfVbXcfZ7ld/ai/PSNV6N8VVVNss9y41w2n9vB6ugovsZoOs3ys80o
n86dMgxP1gDQnLIGgOaUNQA0p6wBoDllDQDNKWsAaE5ZA0BzyhoAmlPWANCcsga

## Who are the best directors?
Obviously we are going to need some shrinkage to to prevent Pixar directors from taking everything.

In [21]:
director_cols = [:id, :director]
directors = readtable("../data/director.csv", names = director_cols, header=false);

In [22]:
director_basics = join(directors, basics_2, on=:id, kind=:left);

In [23]:
director_scores = by(director_basics, :director, df->DataFrame(score = mean(df[:adjusted_average]), 
        user_score = mean(df[:user_score])))
director_scores = director_scores[isna(director_scores[:score]) .== false, :]
director_scores = director_scores[isna(director_scores[:user_score]) .== false, :]
sort!(director_scores, cols=[:score], rev=true)[1:10, :]

,director,score,user_score
1,Cristian Mungiu,9.050377400786404,7.9
2,Jan Pinkava,9.026181639964921,8.6
3,Ronaldo Del Carmen,8.944239453886418,8.7
4,Maren Ade,8.828465136872255,7.0
5,Andrey Zvyagintsev,8.767335551723594,7.3
6,Damien Chazelle,8.734170181101062,8.65
7,Jules Dassin,8.71621170738931,8.3
8,Florian Henckel von Donnersmarck,8.70186804730541,8.9
9,Andrew Jarecki,8.678789111314432,8.0
10,Fritz Lang,8.622580857412743,8.3


Obviously again a problem of not enough movies. Since we're doing this for the second time, let's factor out the code we used the first time.

In [24]:
director_reviews = join(director_basics, reviews, on=:id, kind=:inner);

In [25]:
model_2 = fit_bayes(director_reviews, :director, :stars);

In [26]:
director_reviews_adj = EmpiricalBayes.return_posterior(director_reviews, :director, :stars, model_2);

In [27]:
sort!(director_reviews_adj, cols=[:adjusted_average], rev=true)
director_reviews_adj[1:20, [:director]]

,director
1,Cristian Mungiu
2,Jan Pinkava
3,Ronaldo Del Carmen
4,Damien Chazelle
5,Barry Jenkins
6,Jean-Pierre Melville
7,Maren Ade
8,Andrey Zvyagintsev
9,Florian Henckel von Donnersmarck
10,Lee Unkrich


Still not good. This comes from the fact that we are using as a data point, a review. Let's use as a data point, a movie. We need to do empirical Bayes fun stuff on movies as well.

Let's reprocess the data so as to have one line per movie.

In [50]:
director_basics

,id,director,title,metascore,user_score,release_date,running_time,rating,company,positive,mixed,negative,adjusted_average,isnew
1,10-cent-pistol,Michael C. Martin,10 Cent Pistol,37,6.5,"July 24, 2015",91 min,"['Rated R for violence', 'language throughout', 'some sexual references and drug use (rating surrendered)']",Route 17 Entertainment,2,1,1,4.935719426752363,true
2,10-cloverfield-lane,Dan Trachtenberg,10 Cloverfield Lane,76,7.7,"March 11, 2016",104 min,"['Rated PG-13 for thematic material including frightening sequences of threat with some violence', 'and brief language']",Paramount Pictures,632,89,50,7.543871241462177,true
3,10-items-or-less,Brad Silberling,10 Items or Less,54,5.8,"December 1, 2006",82 min,Rated R for language.,Revelations Entertainment,13,5,6,5.879600631192561,true
4,10-things-i-hate-about-you,Gil Junger,10 Things I Hate About You,70,6.9,"March 31, 1999",97 min,"['Rated PG-13 for crude sex-related humor and dialogue', 'alcohol and drug-related scenes', 'all involving teens.']",Touchstone Pictures,142,35,20,6.925524233025344,false
5,10-years,Jamie Linden,10 Years,61,6.5,"September 14, 2012",100 min,"['Rated R for sexual content', 'and language throughout.']",Temple Hill Entertainment,10,2,3,6.227086358485928,true
6,1000-times-good-night,Erik Poppe,"1,000 Times Good Night",57,6.0,"October 24, 2014",117 min,Not Rated,Film i Väst,3,0,1,6.0406667381434005,true
7,10000-bc,Roland Emmerich,"10,000 BC",34,4.6,"March 7, 2008",109 min,Rated PG-13 for sequences of intense action and violence.,Warner Bros. Pictures,89,85,120,4.141911434152098,true
8,10000-km,Carlos Marques-Marcet,"10,000 km",75,7.3,"July 10, 2015",99 min,"['Rated R for some strong sexual content including dialogue', 'language and brief graphic nudity']",Televisión Española (TVE),6,2,0,7.106627727225048,true
9,101-dalmatians,Stephen Herek,101 Dalmatians,49,5.8,"November 27, 1996",103 min,G,Walt Disney Pictures,28,64,11,5.715191533377097,false
10,101-reykjavik,Baltasar Kormákur,Reykjavík,68,7.7,"July 25, 2001",88 min,Not Rated,Zentropa Entertainments,13,0,3,6.676204798536832,false


## Non-negative Matrix Factorization for similarity between user and critic.
The goal here is to:
- give a certain number of movies to the user to rate
- identify the critic the user is the closest to
- reccomend movies to the user based on that critic's preferences.

This is better (I think) than traditional Collaborative Filtering, because the standard user won't see as many movies as the critics - so this could allow recommending more obscure movies, and also suggest to the user to read that critic's paper in the future...

We could imagine having a hierarchical improvement, where for each genre, we would select a critic. But maybe later.

In [28]:
film_ids = convert(Array, unique(reviews[:id]))
critic_ids = convert(Array, unique(reviews[:critic]));
film_dict = Dict(collect(zip(film_ids, 1:length(film_ids))))
critic_dict = Dict(collect(zip(critic_ids, 1:length(critic_ids))));
film_is = [film_dict[film] for film in reviews[:id]]
critic_is = [critic_dict[critic] for critic in reviews[:critic]];

In [29]:
critic_x_film = sparse(critic_is, film_is, reviews[:score])

2707×8297 sparse matrix with 175327 Int64 nonzero entries:
	[1   ,    1]  =  70
	[2   ,    1]  =  65
	[3   ,    1]  =  60
	[4   ,    1]  =  50
	[5   ,    1]  =  40
	[6   ,    1]  =  30
	[7   ,    1]  =  12
	[8   ,    2]  =  91
	[9   ,    2]  =  91
	[10  ,    2]  =  88
	⋮
	[2074, 8296]  =  88
	[2707, 8296]  =  75
	[21  , 8297]  =  80
	[62  , 8297]  =  25
	[66  , 8297]  =  50
	[67  , 8297]  =  75
	[71  , 8297]  =  42
	[91  , 8297]  =  50
	[108 , 8297]  =  30
	[162 , 8297]  =  70
	[225 , 8297]  =  60

In [30]:
means = Array{Float64}(size(critic_x_film, 2)) 
for i in 1:size(critic_x_film, 2)
   means[i] = mean(nonzeros(critic_x_film[:, i]))
end

In [31]:
fit_mle(Dirichlet, [.1 .1; .9 .9], init= [.1 .1])

LoadError: TypeError: #fit_mle: in typeassert, expected Array{Float64,1}, got Array{Float64,2}